In [2]:
import SimpleITK as sitk
import numpy as np
from scipy.ndimage import distance_transform_edt, binary_erosion

ct = sitk.ReadImage("/home/abigail/mphys/Total_Segmentator/1Zr/CT_WHOLE_CNS_cropped.nii")
seg = sitk.ReadImage("/home/abigail/mphys/Total_Segmentator/1Zr/mandible.nii.gz")

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(ct)
resampler.SetInterpolator(sitk.sitkNearestNeighbor) 
seg_resampled = resampler.Execute(seg)

ct_np = sitk.GetArrayFromImage(ct)
seg_np = sitk.GetArrayFromImage(seg_resampled)

grad = sitk.GradientMagnitudeRecursiveGaussian(ct, sigma=1.0)
grad_np = sitk.GetArrayFromImage(grad)

# Label mask and boundary (e.g. mandible label = 1)
label = 1
mask = (seg_np == label)
boundary = np.logical_and(mask, ~binary_erosion(mask))

# Strong edges (top 10% of gradient values)
thr = np.percentile(grad_np, 90)
strong_edges = grad_np > thr

# making sure shapes match
assert boundary.shape == strong_edges.shape, f"Shape mismatch: {boundary.shape} vs {strong_edges.shape}"

# voxels to mm
spacing = ct.GetSpacing()
dist_to_edge_mm = distance_transform_edt(~strong_edges, sampling=spacing[::-1])

# distances for boundary voxels 
boundary_dists = dist_to_edge_mm[boundary]

if boundary_dists.size == 0:
    raise ValueError(f"No boundary voxels found for label {label}. Check segmentation alignment or label value.")

# errors
mean_d = boundary_dists.mean()
std_d  = boundary_dists.std()
sem_d  = std_d / np.sqrt(boundary_dists.size)
pct_1mm = (boundary_dists <= 1).mean() * 100
pct_2mm = (boundary_dists <= 2).mean() * 100

print(f"CT spacing: {spacing}")
print(f"Structure {label}:")
print(f"  Mean distance: {mean_d:.2f} ± {std_d:.2f} mm (Standard Deviation)")
print(f"  Standard error of mean: ±{sem_d:.3f} mm  (n={boundary_dists.size})")
print(f"  % boundary ≤ 1 mm: {pct_1mm:.1f}%")
print(f"  % boundary ≤ 2 mm: {pct_2mm:.1f}%")


CT spacing: (0.9765625, 0.9765625, 2.0)
Structure 1:
  Mean distance: 0.03 ± 0.19 mm (Standard Deviation)
  Standard error of mean: ±0.002 mm  (n=8888)
  % boundary ≤ 1 mm: 99.3%
  % boundary ≤ 2 mm: 99.9%
